# VIRTUAL MOUSE

##### WXPYTHON
- pip install wxPython
- With wxPython software developers can create truly native user interfaces for their Python applications, that run with little or no modifications on Windows, Macs and Linux or other unix-like systems.
- wxPython is a cross-platform GUI toolkit for the Python programming language. It allows Python programmers to create programs with a robust, highly functional graphical user interface, simply and easily. It is implemented as a set of Python extension modules that wrap the GUI components of the popular wxWidgets cross platform library, which is written in C++.
- https://wxpython.org/Phoenix/docs/html/wx.functions.html

##### PYNPUT
- pip install pynput
- This library allows you to control and monitor input devices. Currently, mouse and keyboard input and monitoring are supported.


### Demo For pynput

In [ ]:
from pynput.mouse import Button, Controller

mouse = Controller()

# Read pointer position
print('The current pointer position is {0}'.format(
    mouse.position))

# Set pointer position
mouse.position = (10, 20)
print('Now we have moved it to {0}'.format(
    mouse.position))

# Move pointer relative to current position
mouse.move(5, -5)

# Press and release
mouse.press(Button.left)
mouse.release(Button.left)

# Double click; this is different from pressing and releasing
# twice on Mac OSX
mouse.click(Button.left, 2)

# Scroll two steps down
mouse.scroll(0, 2)

### Demo For wxPython

In [ ]:
# First things, first. Import the wxPython package.
import wx

# Next, create an application object.
app = wx.App()

# Then a frame.
frm = wx.Frame(None, title="Hello World")

# Show it.
frm.Show()

# Start the event loop.
app.MainLoop()

##### GaussianBlur()
- Blurs an image using a Gaussian filter.The function convolves the source image with the specified Gaussian kernel. In-place filtering is supported.
- dst	=	cv.GaussianBlur(	src, ksize, sigmaX[, dst[, sigmaY[, borderType]]]	)


##### Morphological transformations
Morphological transformations are some simple operations based on the image shape. It is normally performed on binary images. It needs two inputs, one is our original image, second one is called structuring element or kernel which decides the nature of operation. Two basic morphological operators are Erosion and Dilation. Then its variant forms like Opening, Closing, Gradient etc also comes into play.
https://docs.opencv.org/trunk/d9/d61/tutorial_py_morphological_ops.html

##### morphologyEx()
- Performs advanced morphological transformations.The function cv::morphologyEx can perform advanced morphological transformations using an erosion and dilation as basic operations.Any of the operations can be done in-place. In case of multi-channel images, each channel is processed independently.
- dst	=	cv.morphologyEx(	src, op, kernel[, dst[, anchor[, iterations[, borderType[, borderValue]]]]]	)


###### DAMPING
- It is used to make mouse cursor smooth so that it don't flicker everything and to reduce it's speed as by default it is very fast
- adding the damping factor so that the movement of the mouse is smoother

###### Pinch_Flag
It is used because when object/fingers touches each other and become a single object it repeatedly keeps clicking and it creates a problem. so by this we stop repetead clicks if camera detects a single object/finger or distance b/w finguers=0

##### CONTOURS
- Contours can be explained simply as a curve joining all the continuous points (along the boundary), having same color or intensity. The contours are a useful tool for shape analysis and object detection and recognition.
- there are three arguments in cv.findContours() function, first one is source image, second is contour retrieval mode, third is contour approximation method. And it outputs the contours and hierarchy. Contours is a Python list of all the contours in the image. Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object.
- https://docs.opencv.org/trunk/d4/d73/tutorial_py_contours_begin.html

In [ ]:
import cv2 as cv                              #For Computer Vision & Image processing
import numpy as np                            #For numbers
from pynput.mouse import Button,Controller    #I/O controls 
import wx                                     #GUI Toolkit

In [ ]:
mouse = Controller()

In [ ]:
app = wx.App(False)          #Making new window GUI

#app = QtGui.QApplication([])

In [ ]:
(sx,sy) = wx.GetDisplaySize()         #wx.GetDisplaySize() - Returns the display size in pixels.

#sx, sy = screen_resolution.width(), screen_resolution.height()

In [ ]:
#(camx,camy) = (320,240)

In [ ]:
cam = cv.VideoCapture(0)          #Open Camera shutter / Start camera

In [ ]:
(camx,camy) = (320,240)   # - camera will capture an image in this resolution 

#Adjusting Resolution

cam.set(3,camx)     
cam.set(4,camy)

In [ ]:
mlocold = np.array([0,0])
mouseloc = np.array([0,0])
damfac = 2.5                # It's value should be more than 1
pinch_flag = 0

In [ ]:
#mouseLoc=mLocOld+(targetLoc-mLocOld)/damfac
#mouse_location = mouse_location_old + (Target_location - mouse_location_old)/Damping_Factor

#adding the damping factor so that the movement of the mouse is smoother

_____________________________________________________________________________________________________________________________

In [1]:
import cv2 as cv 
import numpy as np 
from pynput.mouse import Button,Controller
import wx

mouse = Controller()
app = wx.App(False)
(sx,sy) = wx.GetDisplaySize()
(camx,camy) = (320,240)

cam = cv.VideoCapture(0)
cam.set(3,camx)
cam.set(4,camy)

mlocold = np.array([0,0])
mouseloc = np.array([0,0])
damfac = 2.5
pinch_flag = 0
outerx,outery,outerw,outerh=(0,0,0,0)




while(1):
    
    #  CAPTURE FRAME-BY-FRAME:
    ret,img = cam.read()
    
    img = cv.GaussianBlur(img,(5,5),0)   #Image filter
    
    #  Convert BGR to HSV
    hsv_img = cv.cvtColor(img,cv.COLOR_BGR2HSV)     #For Object Detection(colour) convert BGR to HSV
    
    #  Create the mask
    mask = cv.inRange(hsv_img,np.array([33,80,40]),np.array([102,255,255]))    #Range of green colour
    
    #  Morphology
    mask_open = cv.morphologyEx(mask,cv.MORPH_OPEN,np.ones((5,5)))
    mask_close = cv.morphologyEx(mask_open,cv.MORPH_CLOSE,np.ones((20,20)))
    mask_final = mask_close

    conts,_ = cv.findContours(mask_final.copy(),cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    cv.drawContours(img,conts,-1,(0,0,255),3)
    
    
    
    
    
    
    
    
    if(len(conts)==2):              #len(conts)==2 open position
        if(pinch_flag==1):          #If fingers are open
            pinch_flag = 0    
            mouse.release(Button.left)
        
        
        x1,y1,w1,h1 = cv.boundingRect(conts[0])   #First finger/object
        x2,y2,w2,h2 = cv.boundingRect(conts[1])   #Second finger/object
        
        
        #  EITHER BUILD RECTANGLE OR CIRCLE
        #cv.rectangle(img,(x1,y1),(x1+w1,y1+h1),(255,0,0),2)
        #cv.rectangle(img,(x2,y2),(x2+w2,y2+h2),(255,0,0),2)   
        #cv.rectangle(input(starting_x,starting_y),(final coordinates),(colour),thickness)  
        #-(x2+w2,y2+h2) as rectangle will constantly move so their coordinates will change according to fingers/object
        
        
        #  MAKING CENTRE (Circle)
        cx1 = round(x1+w1/2)
        cy1 = round(y1+h1/2)
        cx2 = round(x2+w2/2)
        cy2 = round(y2+h2/2)
        
        #  MAKING LINE BETWEEN CENTRE OF FIRST OBJECT & SENCOND OBJECT
        cv.line(img,(cx1,cy1),(cx2,cy2),(255,0,0),2)
        
        #  CENTRE OF LINE DRAWN BETWEEN FIRST OBJECT & SENCOND OBJECT
        cx = round(cx1/2+cx2/2)
        cy = round(cy1/2+cy2/2)
        
        cv.circle(img,(cx,cy),2,(0,0,255),2)   #-Marking the centre by red circle
        
        
        
        mouseloc = mlocold+((cx,cy)-mlocold)/damfac
        #  AS THERE IS A DIFFERENCE B/W SCREEN RESOLUTION & CAMERA RESOLUTION (we want mouse cursor to move on whole screen not only on camera window)
        mouse.position = (round(sx - (mouseloc[0]*sx/camx)),round((mouseloc[1]*sy/camy)))
        #mouse.position=(cx*sx/camx,cy*sy/camy):                                   - this will create mirror effect     
        #mouse.position=(sx-(cx*sx/camx),cy*sy/camy):                              - to correct mirror effect
        #mouse.position=(sx-(mouseloc[0]*sx/camx)),round((mouseloc[1]*sy/camy))    -current mouse location(after smoothning)
        mlocold = mouseloc    #It will change the old location to new one for next itration
        
        #  IF ONE OF FINGERS ABRUBTLY GOT DISAPPEAR THEN WE WILL SE THE AREA 
        outerx,outery,outerw,outerh=cv.boundingRect(np.array([[[x1,y1],[x1+w1,y1+h1],[x2,y2],[x2+w2,y2+h2]]])) 
        cv.rectangle(img,(outerx,outery),(outerx+outerw,outery+outerh),(255,0,0),2)
        #If area is very small that means it is only single finger beacuse when we will perform a click action then combination of both fingers will have large area
    
    
    
    
    
    
    
    
    
    
    elif(len(conts)==1):                      #len(conts)==1 close position fingers/object got close (click)
        x,y,w,h = cv.boundingRect(conts[0])   #On closing fingers it will become a single object/box
        if(pinch_flag==0):                    # If fingers are close
            if(abs((w*h-outerw*outerh)*100/(w*h))<20):
                pinch_flag = 1
                mouse.press(Button.left)
        
       # x,y,w,h = cv.boundingRect(conts[0])   #On closing fingers it will become a single object/box
        
        #  EITHER BUILD RECTANGLE OR CIRCLE
        #cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        
        
        #  ELSE - SO THAT ONLY ONE FINGER/OBJECT IS GUIDING IT WON'T WORK (Right click action won't happen)
        else:
             #  MAKING CENTRE (Circle) - of a single object now
            cx = round(x+w/2)
            cy = round(y+h/2)
        
            # CIRCLE WHEN OBJECTS MEET AND RIGHT CLICK OPERATION OCCURS
            cv.circle(img,(cx,cy),20,(0,0,255),2)
        
            #  TO REMOVE THE REPEATED ITRATION OF CHECKING AS IT IS CHECKED ABOVE
            outerx,outery,outerw,outerh=(0,0,0,0)
        
            mouseloc = mlocold+((cx,cy)-mlocold)/damfac
            #AS THERE IS A DIFFERENCE B/W SCREEN RESOLUTION & CAMERA RESOLUTION (we want mouse cursor to move on whole screen not only on camera window)
            mouse.position = (round(sx - mouseloc[0]*sx/camx),round(mouseloc[1]*sy/camy))
            #mouse.position=(cx*sx/camx,cy*sy/camy):       - this will create mirror effect     
            #mouse.position=(sx-(cx*sx/camx),cy*sy/camy):  - to correct mirror effect
            #mouse.position=(sx-(mouseloc[0]*sx/camx)),round((mouseloc[1]*sy/camy))    -current mouse location(after smoothning)
            mlocold = mouseloc  #It will change the old location to new one for next itration
        
        
        
        
    
    
    
    
    cv.imshow("cam",img)
    #cv.imshow("mask",mask)
    #cv.imshow("mask open",mask_open)
    cv.imshow("mask close",mask_close)
    if cv.waitKey(10) == 13:        #End program with ENTER (13 is key foe enter)
        break
    
cv.destroyAllWindows()

______________________________________________________________________________________________________________________________

In [2]:
cam.release()

pip install pynput

pip install wxpython

pip install opencv-python